# Double-bracket Iteration Scheduling Strategies

This notebook presents the different strategies for scheduling the step durations for the double-bracket iteration algorithm and their resepctive accuracies.

Import the dependencies

In [ ]:
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

from qibo import hamiltonians, set_backend
from qibo.models.dbi.double_bracket import DoubleBracketGeneratorType, DoubleBracketScheduling, DoubleBracketIteration
from qibo.models.dbi.utils import *

## Canonical
Set up the basic test case with the transverse field ising model hamiltonian and the canonical bracket as the generator.

In [ ]:
# Hamiltonian
set_backend("qibojit", "numba")

# hamiltonian parameters
nqubits = 5
h = 3

# define the hamiltonian
H_TFIM = hamiltonians.TFIM(nqubits=nqubits, h=h)

# initialize class
dbi = DoubleBracketIteration(deepcopy(H_TFIM),mode=DoubleBracketGeneratorType.canonical)
print("Initial off diagonal norm", dbi.off_diagonal_norm)

We first generate the relationship between the step duration and the off-diagoanl norm (loss function) for the first step of the iteration.

In [ ]:
# generate data for plotting sigma decrease of the first step
s_space = np.linspace(1e-5, 0.6, 100)
off_diagonal_norm_diff = []
for s in s_space:
    dbi_eval = deepcopy(dbi)
    dbi_eval(s)
    off_diagonal_norm_diff.append(dbi_eval.off_diagonal_norm - dbi.off_diagonal_norm)

The default scheduling strategy is grid search: `DoubleBracketScheduling.use_grid_serach`. This strategy specifies a list of step durations to test one by one and finds the one that maximizes the cost function (off-digonal norm of Hamiltonian)

In [ ]:
# grid_search
step_grid = dbi.choose_step(scheduling=DoubleBracketScheduling.use_grid_search)
print('grid_search step:', step_grid)
# hyperopt
step_hyperopt = dbi.choose_step(scheduling=DoubleBracketScheduling.use_hyperopt, max_evals=100, step_max=0.6)
print('hyperopt_search step:', step_hyperopt)
# polynomial expansion
step_poly = dbi.choose_step(scheduling=DoubleBracketScheduling.use_polynomial_approximation, n=5)
print('polynomial_approximation step:', step_poly)

In [ ]:
# Plot the results
plt.plot(s_space, off_diagonal_norm_diff)
plt.axvline(x=step_grid, color='r', linestyle='-',label='grid_search')
plt.axvline(x=step_hyperopt, color='g', linestyle='--',label='hyperopt')
plt.axvline(x=step_poly, color='m', linestyle='-.',label='polynomial')
plt.ylabel(r'$||\sigma(H_0)||-\sigma(H_k)||$')
plt.xlabel('s')
plt.title('hyperopt first step')
plt.legend()
print('The minimum for cost function in the tested range is:', step_grid)

## Specified diagonal operator

While for the cannonical case, all the scheduling methods are accurate, it is important to realize that the global minimum of the loss function is not always so obvious. It is thus necessary to show whether the 3 converges to an agreeable step duration using different iteration generators, such as the Pauli 'ZZ..Z' operator and 'ZZ..I' operator.

In [ ]:
# Generate the digaonal operators
Z_op = SymbolicHamiltonian(str_to_symbolic("Z"*nqubits)).dense.matrix
ZI_op = SymbolicHamiltonian(str_to_symbolic("Z"*(nqubits-1)+"I")).dense.matrix

In [ ]:
dbi = DoubleBracketIteration(deepcopy(H_TFIM),mode=DoubleBracketGeneratorType.single_commutator)
d = Z_op
# generate data for plotting sigma decrease of the first step
s_space = np.linspace(1e-5, 0.6, 100)
off_diagonal_norm_diff = []
for s in s_space:
    dbi_eval = deepcopy(dbi)
    dbi_eval(s,d=d)
    off_diagonal_norm_diff.append(dbi_eval.off_diagonal_norm - dbi.off_diagonal_norm)

In [ ]:
# grid_search
step_grid = dbi.choose_step(scheduling=DoubleBracketScheduling.use_grid_search, step_max=0.6, d=d)
print('grid_search step:', step_grid)
# hyperopt
step_hyperopt = dbi.choose_step(scheduling=DoubleBracketScheduling.use_hyperopt, d=d, max_evals=100, step_max=0.6)
print('hyperopt_search step:', step_hyperopt)
# polynomial expansion
step_poly = dbi.choose_step(scheduling=DoubleBracketScheduling.use_polynomial_approximation, d=d, n=5)
print('polynomial_approximation step:', step_poly)

In [ ]:
# Plot the results
plt.plot(s_space, off_diagonal_norm_diff)
plt.axvline(x=step_grid, color='r', linestyle='-',label='grid_search')
plt.axvline(x=step_hyperopt, color='g', linestyle='--',label='hyperopt')
plt.axvline(x=step_poly, color='m', linestyle='-.',label='polynomial')
plt.ylabel(r'$||\sigma(H_0)||-\sigma(H_k)||$')
plt.xlabel('s')
plt.title('hyperopt first step')
plt.legend()
print('The minimum for cost function in the tested range is:', step_grid)

We see that there are two similar "minimal point" at 0.03 and 0.22, with the latter being the absolute minimum by an insignificant advantage. However, for practical reasons, we prefer taking the first close-minimum calculated by polynomial approximation. Hence, we can use the polynomial approximation to restrict the search area and obtain better results. For example, we define a search range of 0.1 around the polynomial step.

## Use polynomial expansion as an restriction of hyperopt/grid range

In [ ]:
search_range = 0.1
if step_poly < search_range/2:
    step_min = 0
    step_max = search_range
else:
    step_min = step_poly - search_range/2
    step_max = step_poly + search_range/2
# grid_search
step_grid = dbi.choose_step(scheduling=DoubleBracketScheduling.use_grid_search, step_min=step_min, step_max=step_max, d=d)
print('grid_search step:', step_grid)
# hyperopt
step_hyperopt = dbi.choose_step(scheduling=DoubleBracketScheduling.use_hyperopt, step_min=step_min, step_max=step_max, max_evals=100, d=d,)
print('hyperopt_search step:', step_hyperopt)

In [ ]:
# Plot the results
plt.plot(s_space, off_diagonal_norm_diff)
plt.axvline(x=step_grid, color='r', linestyle='-',label='grid_search')
plt.axvline(x=step_hyperopt, color='g', linestyle='--',label='hyperopt')
plt.axvline(x=step_poly, color='m', linestyle='-.',label='polynomial')
plt.ylabel(r'$||\sigma(H_0)||-\sigma(H_k)||$')
plt.xlabel('s')
plt.title('hyperopt first step')
plt.legend()
print('The minimum for cost function in the tested range is:', step_grid)